In [1]:
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import zero_one_loss
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE

In [41]:
### SET HYPERPARAMETERS
GENES_SELECTION = 1000
CNA_SELECTION = 1000

In [3]:
patient_df = pd.read_csv("/home/rimichael/Uni/biohack/bric/data_clinical_patient.txt", sep="\t", skiprows=4)
patient_df = patient_df.set_index('PATIENT_ID')
patient_df

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,AGE_AT_DIAGNOSIS,OS_MONTHS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
PATIENT_ID,,,,,,,,,,,,,,,,,,,,
MB-0000,10.0,6.044,NaN,NO,1.0,Positve,NEUTRAL,YES,Post,4ER+,75.65,140.500000,LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY
MB-0002,0.0,4.020,High,NO,1.0,Positve,NEUTRAL,YES,Pre,4ER+,43.19,84.633333,LIVING,LumA,ER+/HER2- High Prolif,Living,Right,YES,Ductal/NST,BREAST CONSERVING
MB-0005,1.0,4.030,High,YES,1.0,Positve,NEUTRAL,YES,Pre,3,48.87,163.700000,DECEASED,LumB,NaN,Died of Disease,Right,NO,Ductal/NST,MASTECTOMY
MB-0006,3.0,4.050,Moderate,YES,1.0,Positve,NEUTRAL,YES,Pre,9,47.68,164.933333,LIVING,LumB,NaN,Living,Right,YES,Mixed,MASTECTOMY
MB-0008,8.0,6.080,High,YES,1.0,Positve,NEUTRAL,YES,Post,9,76.97,41.366667,DECEASED,LumB,ER+/HER2- High Prolif,Died of Disease,Right,YES,Mixed,MASTECTOMY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTS-T2428,0.0,2.540,NaN,NaN,1.0,Positve,NaN,NaN,NaN,NaN,70.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MTS-T2429,0.0,4.560,NaN,NaN,1.0,Positve,NaN,NaN,NaN,NaN,63.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MTS-T2430,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
sub_patient_df = patient_df[["OS_MONTHS", "OS_STATUS", "INTCLUST","VITAL_STATUS"]]
sub_patient_df

,OS_MONTHS,OS_STATUS,INTCLUST,VITAL_STATUS
PATIENT_ID,,,,
MB-0000,140.500000,LIVING,4ER+,Living
MB-0002,84.633333,LIVING,4ER+,Living
MB-0005,163.700000,DECEASED,3,Died of Disease
MB-0006,164.933333,LIVING,9,Living
MB-0008,41.366667,DECEASED,9,Died of Disease
...,...,...,...,...
MTS-T2428,NaN,NaN,NaN,NaN
MTS-T2429,NaN,NaN,NaN,NaN
MTS-T2430,NaN,NaN,NaN,NaN


In [5]:
cna_df = pd.read_csv("/home/rimichael/Uni/biohack/bric/data_CNA.txt", sep="\t")
cna_df = cna_df.drop(columns="Entrez_Gene_Id")
cna_df = cna_df.T
cna_df.columns = cna_df.iloc[0]
cna_df = cna_df.drop("Hugo_Symbol")
cna_df

Hugo_Symbol,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
MB-0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MB-0039,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
MB-0045,-1,-1,0,-1,-1,-1,-1,0,0,2,...,0,0,0,0,0,1,-1,0,-2,0
MB-0046,0,0,0,-1,-1,-1,-1,0,-1,0,...,0,0,-1,-1,0,0,0,0,-1,0
MB-0048,0,0,1,0,0,0,0,0,-1,0,...,0,1,0,0,0,0,0,1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-6020,1,1,0,0,0,0,0,0,-1,0,...,0,0,-1,-1,0,0,0,0,-1,2
MB-6213,0,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MB-6230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1,-1,0,0,0
MB-7148,0,0,-1,0,0,0,0,-1,0,0,...,0,-1,0,0,0,0,0,0,0,0


In [6]:
# combined dataset
cna_data_df = cna_df.join(sub_patient_df, how="left")
cna_train_df = cna_data_df.drop(columns=["OS_STATUS", "OS_MONTHS", "VITAL_STATUS"]).dropna()
y = cna_train_df["INTCLUST"]
y

MB-0000    4ER+
MB-0039    4ER+
MB-0045    4ER-
MB-0046       5
MB-0048    4ER+
           ... 
MB-7133       1
MB-0156    4ER+
MB-0210    4ER+
MB-0635    4ER+
MB-0876    4ER-
Name: INTCLUST, Length: 1898, dtype: object

In [14]:
cna_train_df = cna_train_df.drop(columns="INTCLUST")

In [15]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols
    
    def transform(self, X):
        col_list = [X[:, c:c+1] for c in self.cols]
        return np.concatenate(col_list, axis=1)
    
    def fit(self, X, y=None):
        return self

In [ ]:
# TODO combine dataset for voting classifier 

In [32]:
expression_df

Hugo_Symbol,RERE,RNF165,CD049690,BC033982,PHF7,CIDEA,PAPD4,AI082173,SLC17A3,SDS,...,BX115874,BX107598,UGCGL1,VPS72,CSMD3,CC2D1A,CB986545,IGSF9,DA110839,FAM71A
MB-0362,8.676978,6.075331,5.453928,4.994525,5.838270,6.397503,7.906217,5.259461,5.702379,6.930741,...,5.271343,5.680321,7.688492,8.084979,5.161796,6.353215,4.836483,7.304643,5.251843,5.049591
MB-0346,9.653589,6.687887,5.454185,5.346010,5.600876,5.246319,8.267256,5.380069,5.521794,6.141689,...,5.942887,5.461069,7.804165,8.349115,5.197392,6.132355,5.316819,7.933324,5.450611,5.316790
MB-0386,9.033589,5.910885,5.501577,5.247467,6.030718,10.111816,7.959291,5.262024,5.689533,6.529312,...,5.174498,5.304030,7.934309,8.406332,8.087722,6.366335,5.466419,7.580336,5.235394,5.461617
MB-0574,8.814855,5.628740,5.471941,5.316523,5.849428,6.116868,9.206376,5.396576,5.439130,6.430102,...,5.116749,5.632249,7.744562,8.310019,5.780062,6.424048,5.193150,6.903654,5.091927,5.227130
MB-0503,9.274265,5.908698,5.531743,5.244094,5.964661,7.828171,8.706646,5.167213,5.417484,6.684893,...,5.402314,5.472185,7.701394,8.137014,5.498185,6.214301,5.274600,6.839417,5.315224,5.027476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,8.131637,9.101942,5.423027,4.939292,5.644587,5.611189,7.798269,5.219962,5.597732,6.583524,...,5.417529,5.484696,7.643929,8.040024,5.456862,6.269748,5.337776,7.288315,5.359223,5.253696
MB-5453,9.606915,7.427494,5.534115,5.062191,5.927409,5.927031,8.520545,5.129501,5.550549,5.841476,...,5.566320,5.538543,7.048923,7.560101,5.397010,7.088676,5.216496,7.248336,5.544276,5.436415
MB-5471,9.049296,6.850000,5.339346,5.166765,6.117095,6.374305,8.499637,4.961279,5.497546,6.351428,...,5.484182,5.386238,7.733413,7.941895,5.415928,6.110477,5.427567,7.596215,5.405179,5.094339
MB-5127,8.858622,6.550450,5.566071,5.140141,5.936371,5.963092,9.320207,5.408996,5.690297,7.280037,...,5.403071,5.436583,7.311774,7.866579,5.242482,6.316304,4.977467,6.620605,5.631662,5.350708


In [39]:
print("CNA dim:")
print(cna_df.shape)

total_data_df = cna_df.join(patient_df["INTCLUST"], how="left")
total_data_df = total_data_df.join(expression_df, how="inner", rsuffix="_expr")

print("CNA-expression join dim:")
print(total_data_df.shape)

#X_train, X_test, y_train, y_test
y_total = total_data_df["INTCLUST"]
X_total = total_data_df.drop(columns="INTCLUST")

print("X total dim:")
print(X_total.shape)

X_total

CNA dim:
(2173, 22544)
CNA-expression join dim:
(1904, 46913)
X total dim:
(1904, 46912)


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,BX115874,BX107598,UGCGL1,VPS72_expr,CSMD3_expr,CC2D1A_expr,CB986545,IGSF9_expr,DA110839,FAM71A_expr
MB-0000,0,0,0,0,0,0,0,0,0,0,...,5.329883,5.745324,7.021679,8.010657,5.299815,6.235804,5.349555,5.947404,5.615147,5.133576
MB-0039,0,0,0,0,0,0,0,0,0,0,...,5.951742,5.928352,7.926861,7.331212,5.450666,5.985447,5.458363,5.390357,5.706905,5.498739
MB-0045,-1,-1,0,-1,-1,-1,-1,0,0,2,...,5.379279,5.761502,7.807015,8.301804,5.313254,6.281194,5.390784,6.264153,5.426428,5.547820
MB-0046,0,0,0,-1,-1,-1,-1,0,-1,0,...,5.204814,5.667425,7.715507,8.488727,5.307310,6.071653,5.264978,7.837516,5.321490,5.390555
MB-0048,0,0,1,0,0,0,0,0,-1,0,...,5.520178,5.582338,7.194426,7.496171,5.176560,6.065291,5.180065,5.651414,5.358560,5.825701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5332,0,0,0,1,1,1,1,0,0,0,...,5.572049,5.343389,7.108151,7.946166,5.292774,6.936039,5.287442,6.955749,5.526576,5.387808
MB-5417,0,0,0,0,0,0,0,0,0,0,...,5.971279,5.703873,7.751204,8.122284,5.251913,6.293356,5.397620,7.776947,5.411945,5.647204
MB-5614,0,0,0,0,0,0,0,0,0,1,...,5.531173,5.549395,7.463161,8.079911,5.612112,6.522952,5.302244,6.880750,5.416195,5.396307
MB-6329,0,0,0,0,0,0,0,0,0,0,...,5.644017,5.478255,7.114368,8.847109,5.222065,6.350829,5.286291,6.608030,5.476758,5.570788


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.22, random_state=1234)

In [ ]:
cna_clf = RFE(RandomForestClassifier(max_depth= 25, max_leaf_nodes= 30, 
                                       n_estimators= 300, n_jobs=6),
               CNA_SELECTION, step=1)

cna_clf.fit(X_cna_train, y_cna_train)
print(cna_clf.score(X_cna_test, y_cna_test))

In [ ]:
cna_clf.feature_importance_

In [ ]:
gene_expr_pipeline = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(22543, 46911))),
    # TODO extract gene column ranges into constructor
    ('clf', RFE(RandomForestClassifier(max_depth= 25, max_leaf_nodes= 30, 
                                       n_estimators= 300, n_jobs=6),
               GENES_SELECTION, step=1))
])

gene_expr_pipeline.fit(X_train, y_train)
print(gene_expr_pipeline.score(X_test, y_test))

In [ ]:
# TODO implement gridsearch for weights
ensemble_clf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)],
                               voting='soft', weights=[1, 0.5])
ensemble_clf.fit(X_train, y_train)

In [ ]:
print("Training Pipeline Score:")
print(ensemble_clf.score(X_test, y_test))

y_hat = ensemble_clf.predict(X_test)
print("\n\n Test Zero-one loss:")
print(zero_one_loss(y_test, y_hat))

In [16]:
# classify INTCLUST
X_cna_train, X_cna_test, y_cna_train, y_cna_test = train_test_split(cna_train_df, y, test_size=0.22, random_state=1234)
X_cna_train

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
MB-5300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,0
MB-4648,0,0,1,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,2,0
MB-0899,-1,-1,-1,0,0,0,0,-1,-1,0,...,0,-1,0,0,0,-1,-1,0,0,-1
MB-4317,0,0,0,-1,-1,-1,-1,0,0,0,...,0,0,0,0,1,0,0,-1,-1,0
MB-4333,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-6168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MB-5548,-1,-1,-1,1,1,1,1,1,-1,1,...,-1,-1,-1,-1,-1,1,1,1,-1,1
MB-7007,1,1,-1,0,0,0,0,0,0,2,...,-1,2,-1,-1,2,0,0,0,-1,0
MB-4966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,0


In [17]:
# gridsearch parameters
parameters={'n_estimators': [50, 100, 300, 500],
           'max_depth': [2, 10, 25],
           'max_leaf_nodes': [3, 5, 10, 20, 30]}

In [18]:
rf_clf = RandomForestClassifier()
gs_clf = GridSearchCV(rf_clf, parameters, n_jobs=8)

In [20]:
gs_clf.fit(X_cna_train, y_cna_train)
gs_clf.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
             

In [27]:
# see optimal grid search results
for mean, std, params in zip(gs_clf.cv_results_.get('mean_test_score'), 
                             gs_clf.cv_results_.get('std_test_score'), 
                             gs_clf.cv_results_.get('params')):
    print(mean, std*2, params)

0.46891891891891896 0.07589254638972416 {'max_depth': 2, 'max_leaf_nodes': 3, 'n_estimators': 50}
0.47702702702702704 0.0657040312494228 {'max_depth': 2, 'max_leaf_nodes': 3, 'n_estimators': 100}
0.47837837837837843 0.059459459459459484 {'max_depth': 2, 'max_leaf_nodes': 3, 'n_estimators': 300}
0.48243243243243245 0.0592748978363819 {'max_depth': 2, 'max_leaf_nodes': 3, 'n_estimators': 500}
0.4763513513513514 0.05717350216569237 {'max_depth': 2, 'max_leaf_nodes': 5, 'n_estimators': 50}
0.48310810810810806 0.050018258171139814 {'max_depth': 2, 'max_leaf_nodes': 5, 'n_estimators': 100}
0.48243243243243245 0.047470724514189995 {'max_depth': 2, 'max_leaf_nodes': 5, 'n_estimators': 300}
0.47905405405405405 0.06556491605788217 {'max_depth': 2, 'max_leaf_nodes': 5, 'n_estimators': 500}
0.4817567567567568 0.0803571283638838 {'max_depth': 2, 'max_leaf_nodes': 10, 'n_estimators': 50}
0.4817567567567568 0.05581588223800916 {'max_depth': 2, 'max_leaf_nodes': 10, 'n_estimators': 100}
0.481081081081

In [22]:
y_hat_train = gs_clf.predict(X_train)
y_hat = gs_clf.predict(X_test)

print("Training Loss:")
print(zero_one_loss(y_train, y_hat_train))
print("\nZero One Loss Testing")
print(zero_one_loss(y_test, y_hat))

NameError: name 'X_train' is not defined

In [28]:
# take expression into account
expression_df = pd.read_csv("/home/rimichael/Uni/biohack/bric/data_expression_median.txt",sep="\t")
expression_df = expression_df.set_index("Hugo_Symbol")
expression_df = expression_df.drop(columns="Entrez_Gene_Id").T

In [29]:
expression_df

Hugo_Symbol,RERE,RNF165,CD049690,BC033982,PHF7,CIDEA,PAPD4,AI082173,SLC17A3,SDS,...,BX115874,BX107598,UGCGL1,VPS72,CSMD3,CC2D1A,CB986545,IGSF9,DA110839,FAM71A
MB-0362,8.676978,6.075331,5.453928,4.994525,5.838270,6.397503,7.906217,5.259461,5.702379,6.930741,...,5.271343,5.680321,7.688492,8.084979,5.161796,6.353215,4.836483,7.304643,5.251843,5.049591
MB-0346,9.653589,6.687887,5.454185,5.346010,5.600876,5.246319,8.267256,5.380069,5.521794,6.141689,...,5.942887,5.461069,7.804165,8.349115,5.197392,6.132355,5.316819,7.933324,5.450611,5.316790
MB-0386,9.033589,5.910885,5.501577,5.247467,6.030718,10.111816,7.959291,5.262024,5.689533,6.529312,...,5.174498,5.304030,7.934309,8.406332,8.087722,6.366335,5.466419,7.580336,5.235394,5.461617
MB-0574,8.814855,5.628740,5.471941,5.316523,5.849428,6.116868,9.206376,5.396576,5.439130,6.430102,...,5.116749,5.632249,7.744562,8.310019,5.780062,6.424048,5.193150,6.903654,5.091927,5.227130
MB-0503,9.274265,5.908698,5.531743,5.244094,5.964661,7.828171,8.706646,5.167213,5.417484,6.684893,...,5.402314,5.472185,7.701394,8.137014,5.498185,6.214301,5.274600,6.839417,5.315224,5.027476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,8.131637,9.101942,5.423027,4.939292,5.644587,5.611189,7.798269,5.219962,5.597732,6.583524,...,5.417529,5.484696,7.643929,8.040024,5.456862,6.269748,5.337776,7.288315,5.359223,5.253696
MB-5453,9.606915,7.427494,5.534115,5.062191,5.927409,5.927031,8.520545,5.129501,5.550549,5.841476,...,5.566320,5.538543,7.048923,7.560101,5.397010,7.088676,5.216496,7.248336,5.544276,5.436415
MB-5471,9.049296,6.850000,5.339346,5.166765,6.117095,6.374305,8.499637,4.961279,5.497546,6.351428,...,5.484182,5.386238,7.733413,7.941895,5.415928,6.110477,5.427567,7.596215,5.405179,5.094339
MB-5127,8.858622,6.550450,5.566071,5.140141,5.936371,5.963092,9.320207,5.408996,5.690297,7.280037,...,5.403071,5.436583,7.311774,7.866579,5.242482,6.316304,4.977467,6.620605,5.631662,5.350708


In [ ]:
## for each column take out the 4 highest correlated genes
#gene_expr_df = gene_corr.copy()
#gene_list = []
#for gene in list(gene_corr):
#    # keep the informations
#    gene_list.append(gene)
#    try:
#        corr_genes = gene_corr.nlargest(8, gene).index.to_list()
#    catch KeyError as e:
#        # if we run over genes that we have sorted out
#        continue
#    gene_expr_df = gene_expr_df.drop(columns=corr_genes)


In [ ]:
# do recursive feature elimination for a tree fit on gene expression data
gene_data_df = expression_df.join(patient_df["INTCLUST"], how="left")
gene_X_df = gene_data_df.drop(columns="INTCLUST")
gene_y_df = gene_data_df["INTCLUST"]

X_gene_train, X_gene_test, y_gene_train, y_gene_test = train_test_split(gene_X_df, 
                                                                        gene_y, test_size=0.22, random_state=1234)

rf_gene_clf = RandomForestClassifier(n_estimators=100)
rf_gene_clf.fit(X_gene_train, y_gene_train, n_jobs=6)


In [ ]:
y_hat = rf_gene_clf.predict(X_gene_test)
zero_one_loss(y_gene_test, y_hat)

In [ ]:
gene_expr_df

In [ ]:
# pick out highly correlated genes